# Find True North

The objective of this notebook is to determine the true north direction on two aerial photographs given their centerpoint geolocations.

## Imports

In [2]:
import numpy as np
import pandas as pd

import math
import statistics
import os
import cv2

from PIL import Image, ImageDraw
from geographiclib.geodesic import Geodesic

## Code

In [10]:
class FindNorth():

    def detect_and_describe(self, image, alg = "ORB"):
        """
        This function is modified from https://www.pyimagesearch.com/2016/01/11/opencv-panorama-stitching/
        SIFT algorithm is proprietary and can only be used with opencv-python 3.4.2.17 and opencv-contrib-python 3.4.2.17
        or earlier.  ORB is the default because it is fast and open source.
        
        inputs:
        image: cv2.image object to have keypoints detected.
        alg: which algorithm to use, can be "SIFT", "ORB", or "BRISK"
        
        outputs:
        kps: keypoint locations
        features: feature vectors of keypoints
        """
        
        if alg == "SIFT":
            descriptor = cv2.xfeatures2d.SIFT_create()
        elif alg == "ORB":
            descriptor = cv2.ORB_create()
        elif alg == "BRISK":
            descriptor = cv2.BRISK_create()
        (kps, features) = descriptor.detectAndCompute(image, None)
        kps = np.float32([kp.pt for kp in kps])

        return kps, features
    
    def match_keypoints(self, kpsA, kpsB, featuresA, featuresB, ratio, reprojThresh):
        """
        Modified from https://www.pyimagesearch.com/2016/01/11/opencv-panorama-stitching/.
        returns the matching keypoints and homography matrix based on feature vectors returned from detect_and_describe.
        
        inputs:
        kpsA: list keypoints from image A
        kpsB: list keypoints from image B
        featuresA: features from image A
        featuresB: features from image B
        ratio: lowe's ratio test ratio
        reprojThresh: threshold number of keypoint matches for reprojection to occur
        
        outputs:
        matches: list of matching keypoints
        H: homography matrix
        status: status of homography matrix search
        """
        # compute the raw matches and initialize the list of actual matches
        matcher = cv2.DescriptorMatcher_create("BruteForce")
        rawMatches = matcher.knnMatch(featuresA, featuresB, 2)
        matches = []

        # loop over the raw matches
        for m in rawMatches:
            # ensure the distance is within a certain ratio of each other (i.e. Lowe's ratio test)
            if len(m) == 2 and m[0].distance < m[1].distance * ratio:
                matches.append((m[0].trainIdx, m[0].queryIdx))

        # computing a homography requires at least 4 matches
        if len(matches) > 4:
            # construct the two sets of points
            ptsA = np.float32([kpsA[i] for (_, i) in matches])
            ptsB = np.float32([kpsB[i] for (i, _) in matches])

            # compute the homography between the two sets of points
            (H, status) = cv2.findHomography(ptsA, ptsB, cv2.RANSAC, reprojThresh)

            # return the matches along with the homograpy matrix and status of each matched point
            return (matches, H, status)

        # otherwise, no homograpy could be computed
        return None

        
    def procrustes(self, X, Y, scaling=True, reflection=False):
        """
        Modified from https://stackoverflow.com/questions/18925181/procrustes-analysis-with-numpy
        A port of MATLAB's `procrustes` function to Numpy.

        Procrustes analysis determines a linear transformation (translation,
        reflection, orthogonal rotation and scaling) of the points in Y to best
        conform them to the points in matrix X, using the sum of squared errors
        as the goodness of fit criterion.

        inputs:
        X: matrix of input coordinates
        Y: matrix of target coordinates.  Must have equal number of points (rows) to X, but may have 
        fewer dimensions than X
        scaling: if False, the scaling component of the transformation is forced to 1.
        reflection: if 'best' (default), the transformation solution may or may not include a reflection 
        component, depending on which fits the data best. setting reflection to True or False forces a 
        solution with reflection or no reflection respectively.

        outputs
        tform: a dict specifying the rotation, translation and scaling that maps X --> Y

        """

        n,m = X.shape
        ny,my = Y.shape
        muX = X.mean(0)
        muY = Y.mean(0)
        X0 = X - muX
        Y0 = Y - muY
        ssX = (X0**2.).sum()
        ssY = (Y0**2.).sum()

        # centred Frobenius norm
        normX = np.sqrt(ssX)
        normY = np.sqrt(ssY)
        # scale to equal (unit) norm
        X0 /= normX
        Y0 /= normY

        if my < m:
            Y0 = np.concatenate((Y0, np.zeros(n, m-my)),0)

        # optimum rotation matrix of Y
        A = np.dot(X0.T, Y0)
        U,s,Vt = np.linalg.svd(A,full_matrices=False)
        V = Vt.T
        T = np.dot(V, U.T)

        if reflection is not 'best':
            # does the current solution use a reflection?
            have_reflection = np.linalg.det(T) < 0
            # if that's not what was specified, force another reflection
            if reflection != have_reflection:
                V[:,-1] *= -1
                s[-1] *= -1
                T = np.dot(V, U.T)

        traceTA = s.sum()

        if scaling:
            # optimum scaling of Y
            b = traceTA * normX / normY
            # standarised distance between X and b*Y*T + c
            d = 1 - traceTA**2
            # transformed coords
            Z = normX*traceTA*np.dot(Y0, T) + muX
        else:
            b = 1
            d = 1 + ssY/ssX - 2 * traceTA * normY / normX
            Z = normY*np.dot(Y0, T) + muX

        # transformation matrix
        if my < m:
            T = T[:my,:]
        c = muX - b*np.dot(muY, T)
        #transformation values 
        tform = {'rotation':T, 'scale':b, 'translation':c}

        return tform
    
    def match_2_images(self, kpsA, kpsB, featuresA, featuresB, num_matches, ratio, reprojThresh):
        """
        Returns the results for procrustes from the keypoints and features of two images.
        
        inputs:
        kpsA: list keypoint locations of image A
        kpsB: list keypoint locations of image B
        featuresA: list feature vectors describimg kpsA from imageA
        featuresB: list feature vectors describing kpsB from imageB
        num_matches: int number of to be considerded for tform calculation
        ratio: ratio for lowe's ratio test
        reprojThresh: threshold number of keypoint matches for reprojection to occur
        
        outputs:
        tform: a dict specifying the rotation, translation and scaling that maps X --> Y
        
        """
        matches, H, status = self.match_keypoints(kpsA, kpsB, featuresA, featuresB, ratio, reprojThresh)
        if len(matches) >= num_matches:
            matchesA = []
            matchesB = []
            for match in matches:
                matchesA.append(list(kpsA[match[1]]))
                matchesB.append(list(kpsB[match[0]]))
            matchesA = np.array(matchesA)
            matchesB = np.array(matchesB)
            tform = self.procrustes(matchesA, matchesB, True, False)
            return tform
        else:
            return None
        
        
        
    def find_north(self, imageA_loc, imageB_loc, imageA_centerpoint, imageB_centerpoint, resize_ratio=6, alg="ORB", ratio=0.75, reprojThresh=4, num_matches=4):
        """
        
        
        """
        imageA_Lat, imageA_Lon = imageA_centerpoint[0], imageA_centerpoint[1]
        imageB_Lat, imageB_Lon = imageB_centerpoint[0], imageB_centerpoint[1]
        
        imageA = cv2.imread(imageA_loc)
        imageB = cv2.imread(imageB_loc)
        
        #Resize large images to compute keypoints more quickly
        imageA = cv2.resize(imageA, (int(imageA.shape[1]//resize_ratio),int(imageA.shape[0]//resize_ratio)))
        imageB = cv2.resize(imageB, (int(imageB.shape[1]//resize_ratio),int(imageB.shape[0]//resize_ratio)))
        
        imageA_height, imageA_width = imageA.shape[0:2]
        imageB_height, imageB_width = imageB.shape[0:2]
        
        #Detect and describe keypoints
        kpsA, featuresA = self.detect_and_describe(imageA, alg)
        kpsB, featuresB = self.detect_and_describe(imageB, alg)
        
        tform = self.match_2_images(kpsA, kpsB, featuresA, featuresB, num_matches, ratio, reprojThresh)
        
        headingA = Geodesic.WGS84.Inverse(imageB_Lat, imageB_Lon, imageA_Lat, imageA_Lon)['azi2']
        headingB = Geodesic.WGS84.Inverse(imageB_Lat, imageB_Lon, imageA_Lat, imageA_Lon)['azi1']
        
        theta2 = math.degrees(math.atan(tform['translation'][1]/tform['translation'][0]))
        
        ansA = round(-(headingA) + theta2, 2)
        ansB = round(-(headingB) + theta2 + math.degrees(math.asin(tform['rotation'][0,1])),2)
        
        
        
        if tform:
            print('ImageA, true North: ', round(90-ansA,2))
            print('ImageB, true North: ', round(90-ansB,2))
            return tform
        else:
            raise Exception("Images could not be made into a collage.")
            
            
        

## Demo

In [ ]:
imageA_loc = '../demo/find_north/images/c-300/raw/c-300_a-1.tif'
imageB_loc = '../demo/find_north/images/c-300/raw/c-300_a-2.tif'
FN = FindNorth()
tform = FN.find_north(imageA_loc, imageB_loc, (34.8128,-118.8884), (34.8051,-118.8871), alg = "SIFT")